# SWMAL Exercise


## Hyperparameters and Gridsearch 

Machine learning models have certain global parameters which decide on the inner workings of the model. An example of this could be a degree of polynomial models, or number of neurons or hidden layers in neural network models. Choosing the optimal hyperparameters for machine learning models manually is extremely time consuming, since it would involve a silly amount of trial and error. In this exercise we will delve into optimizing the hyperparameters using GridSearch and RandomizedSearch.

### Qa Explain GridSearchCV

The following python code block sets up our functions to load and set up the data, as well as display results of a gridsearch. See detailed explanation in the comments.


In [5]:
# Explanation:
# This block of code loads the data and defines functions which will be used to present results of gridsearch
# GetBestModelCTOR() returns a string with a constructor of the model with the best parameters in it
# SearchReport() displays the best models name, its best parameters, score and index. It also asserts that the scoring system used is f1_micro.
# ClassificationReport() uses the model to predict with the test data supplied in parameters. It then compares the prediction with true values.
# TryKerasImport() asserts that keras module is loaded and ready to be used
# LoadAndSetupData() loads the data and reshapes it if needed, chosen by the parameter 'mode' - either iris, mnist or moon dataset

from time import time
import numpy as np
import sys

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

# Need this to import libitmal on MY Windows machine. Replace with your GITMAL directory or uncomment if you already have gitmal in pythonpath  - Marcin
sys.path.append("C:\\UNI_2023\\ml\\gitmal")

from libitmal import dataloaders as itmaldataloaders # Needed for load of iris, moon and mnist

currmode="N/A" # GLOBAL var!

def SearchReport(model): 
    
    # This method 
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            ret_str=""          
            for key in sorted(best_params):
                value = best_params[key]
                temp_str = "'" if str(type(value))=="<class 'str'>" else ""
                if len(ret_str)>0:
                    ret_str += ','
                ret_str += f'{key}={temp_str}{value}{temp_str}'  
            return ret_str          
        try:
            param_str = GetParams(best_params)
            return type(model).__name__ + '(' + param_str + ')' 
        except:
            return "N/A(1)"
        
    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")
    
    global currmode                
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"   
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_ 

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)                 
    print(classification_report(y_true, y_pred, target_names=target_names))
    print()
    
def FullReport(model, X_test, y_test, t):
    print(f"SEARCH TIME: {t:0.2f} sec")
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)    
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel
    
def LoadAndSetupData(mode, test_size=0.3):
    assert test_size>=0.0 and test_size<=1.0
    
    def ShapeToString(Z):
        n = Z.ndim
        s = "("
        for i in range(n):
            s += f"{Z.shape[i]:5d}"
            if i+1!=n:
                s += ";"
        return s+")"

    global currmode
    currmode=mode
    print(f"DATA: {currmode}..")
    
    if mode=='moon':
        X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
        itmaldataloaders.MOON_Plot(X, y)
    elif mode=='mnist':
        X, y = itmaldataloaders.MNIST_GetDataSet(load_mode=0)
        if X.ndim==3:
            X=np.reshape(X, (X.shape[0], -1))
    elif mode=='iris':
        X, y = itmaldataloaders.IRIS_GetDataSet()
    else:
        raise ValueError(f"could not load data for that particular mode='{mode}', only 'moon'/'mnist'/'iris' supported")
        
    print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

    assert X.ndim==2
    assert X.shape[0]==y.shape[0]
    assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)    
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=0, shuffle=True
    )
    
    print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
    print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
    print()
    
    return X_train, X_test, y_train, y_test

def TryKerasImport(verbose=True):
    
    kerasok = True
    try:
        import keras as keras_try
    except:
        kerasok = False

    tensorflowkerasok = True
    try:
        import tensorflow.keras as tensorflowkeras_try
    except:
        tensorflowkerasok = False
        
    ok = kerasok or tensorflowkerasok
    
    if not ok and verbose:
        if not kerasok:
            print("WARNING: importing 'keras' failed", file=sys.stderr)
        if not tensorflowkerasok:
            print("WARNING: importing 'tensorflow.keras' failed", file=sys.stderr)

    return ok
    
print(f"OK(function setup" + ("" if TryKerasImport() else ", hope MNIST loads works because it seems you miss the installation of Keras or Tensorflow!") + ")")

OK(function setup)


Gridsearch is a method for tuning the hyperparameters of a model automatically, and GridSearchCV is the scikit-learn class that provides this functionality. To use GridSearchCV you simply supply it with the parameters you want it to test, and with values that you want to check. After running GridSearchCV.fit() on a dataset, gridsearch will go through all the possible combination of hyperparameters with the values supplied to it and compare their scoring using a scoring method of your choice. When that is done, the best parameters and the scores will be available in the GridSearchCV object. 

The following code block performs the actual grid search and displays the results using the functions supplied in the previous block. See the code comments for detailed explanation.

In [6]:
# Setup data
X_train, X_test, y_train, y_test = LoadAndSetupData('iris')  # 'iris', 'moon', or 'mnist'

# This is the model type we will test
model = svm.SVC(
    gamma=0.001
)

# These are the parameters that we want gridsearch to evaluate
# They are setup as a Dict[name, vals] with name always being a string and vals being whatever type the parameter values are
# In this particular example we will compare kernels 'linear' and 'rbf' against each other with 'C' (the regularization parameter) values being 0.1, 1 and 10
# This means the model will be fit 2*3 times 
tuning_parameters = {
    'kernel': ('linear', 'rbf'), 
    'C': [0.1, 1, 10]
}

# This is the number of KFolds that gridsearch's cross-validation strategy will use
CV = 5
# Don't display any debug informations
VERBOSE = 0

# Create gridsearch model with hyperparameters tested specified above
# n_jobs is number of jobs ran in parallel when fitting the model: -1 uses all available processors according to sklearn's documentation
# job is a somewhat ambiguous term so what exactly this means depends on the backend implementation in sklearn

# 'f1_micro' scoring method is defined as the micro-averaged harmonic mean of precision and recall.
# According to https://www.visobyte.com/2023/05/precision-recall-and-f1-score-in-object-detection-how-are-they-calculated.html#:~:text=The%20F1%20Score%20is%20a%20harmonic%20mean%20of,%2A%20%28Precision%20%2A%20Recall%29%20%2F%20%28Precision%20%2B%20Recall%29
# The precision score measures the rate of false positives and the recall score measures how accurately it predicts/detects 
grid_tuned = GridSearchCV(model,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)

# Find the best parameters and measure the time to do so using X_train, y_train from the iris dataset
start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

# Report result. Uses previously defined methods to print data about the model. Also runs the best model to predict (X_test, y_test) validating it.
b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

DATA: iris..
  org. data:  X.shape      =(  150;    4), y.shape      =(  150)
  train data: X_train.shape=(  105;    4), y_train.shape=(  105)
  test data:  X_test.shape =(   45;    4), y_test.shape =(   45)



SEARCH TIME: 2.20 sec

Best model set found on train set:

	best parameters={'C': 1, 'kernel': 'linear'}
	best 'f1_micro' score=0.9714285714285715
	best index=2

Best estimator CTOR:
	SVC(C=1, gamma=0.001, kernel='linear')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.962 (+/-0.093) for {'C': 0.1, 'kernel': 'linear'}
	[ 1]: 0.371 (+/-0.038) for {'C': 0.1, 'kernel': 'rbf'}
	[ 2]: 0.971 (+/-0.047) for {'C': 1, 'kernel': 'linear'}
	[ 3]: 0.695 (+/-0.047) for {'C': 1, 'kernel': 'rbf'}
	[ 4]: 0.952 (+/-0.085) for {'C': 10, 'kernel': 'linear'}
	[ 5]: 0.924 (+/-0.097) for {'C': 10, 'kernel': 'rbf'}

Detailed classification report:
	The model is trained on the full development set.
	The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.94      0.97        18
           2       0.92      1.00      0.96        11

    accuracy                        

The best parameters for our model are C=1 with linear kernel, having a score of 0.97143

### Qb Hyperparameter Grid Search using an SDG classifier

We will now use grid search to tune parameters of a Stochastic Gradient Descent classifier. 

In [8]:
from sklearn import linear_model

sgd = linear_model.SGDClassifier()

# Notice different parameters, check sklearn documentation for SGDClassifier for description of these parameters
tuning_parameters = {
    'loss': ['hinge', 'log_loss', 'perceptron'], 
    'penalty': ['l2', 'l1'],
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 100],
    'tol' : [0.0001, 0.001, 0.01, 0.1]
}

# This part is almost the same as in previous code block, we just swap the model
grid_tuned = GridSearchCV(sgd,
                          tuning_parameters,
                          cv=CV,
                          scoring='f1_micro',
                          verbose=VERBOSE,
                          n_jobs=-1)

start = time()
grid_tuned.fit(X_train, y_train)
t = time() - start

b0, m0 = FullReport(grid_tuned, X_test, y_test, t)
print('OK(grid-search)')

SEARCH TIME: 2.64 sec

Best model set found on train set:

	best parameters={'alpha': 0.01, 'loss': 'hinge', 'penalty': 'l1', 'tol': 0.001}
	best 'f1_micro' score=0.9904761904761905
	best index=53

Best estimator CTOR:
	SGDClassifier(alpha=0.01, penalty='l1')

Grid scores ('f1_micro') on development set:
	[ 0]: 0.733 (+/-0.230) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2', 'tol': 0.0001}
	[ 1]: 0.800 (+/-0.152) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2', 'tol': 0.001}
	[ 2]: 0.771 (+/-0.304) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2', 'tol': 0.01}
	[ 3]: 0.781 (+/-0.245) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l2', 'tol': 0.1}
	[ 4]: 0.895 (+/-0.140) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1', 'tol': 0.0001}
	[ 5]: 0.867 (+/-0.185) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1', 'tol': 0.001}
	[ 6]: 0.771 (+/-0.332) for {'alpha': 0.0001, 'loss': 'hinge', 'penalty': 'l1', 'tol': 0.01}
	[ 7]: 0.886 (+/-0.222) for {'alpha': 

The best model has a staggering score of 0.99048, even better than the SVC model we used previously!

### Qc Hyperparameter Random  Search using an SDG classifier

Now, add code to run a `RandomizedSearchCV` instead.

<img src="https://itundervisning.ase.au.dk/SWMAL/L09/Figs/randomsearch.png" alt="WARNING: could not get image from server."  style="width:350px" >
<small><em>
    <center> Conceptual graphical view of randomized search for two distinct hyperparameters. </center> 
</em></small>

Use these default parameters for the random search, similar to the default parameters for the grid search

```python
random_tuned = RandomizedSearchCV(
    model, 
    tuning_parameters, 
    n_iter=20, 
    random_state=42, 
    cv=CV, 
    scoring='f1_micro', 
    verbose=VERBOSE, 
    n_jobs=-1
)
```

but with the two new parameters, `n_iter` and `random_state` added. Since the search-type is now random, the `random_state` gives sense, but essential to random search is the new `n_tier` parameter.

So: investigate the `n_iter` parameter...in code and write a conceptual explanation  in text.

Comparison of time (seconds) to complete `GridSearch` versus `RandomizedSearchCV`, does not necessarily give any sense, if your grid search completes in a few seconds (as for the iris tiny-data). You need a search that runs for minutes, hours, or days.

But you could compare the best-tuned parameter set and best scoring for the two methods. Is the random search best model close to the grid search?  

In [ ]:
# TODO:
assert False, "implement a random search for the SGD classifier.."

## Qd MNIST Search Quest II

Finally, a search-quest competition: __who can find the best model+hyperparameters for the MNIST dataset?__

You change to the MNIST data by calling `LoadAndSetupData('mnist')`, and this is a completely other ball-game that the iris _tiny-data_: it's much larger (but still far from _big-data_)!

* You might opt for the exhaustive grid search, or use the faster but-less optimal random search...your choice. 

* You are free to pick any classifier in Scikit-learn, even algorithms we have not discussed yet---__except Neural Networks and KNeighborsClassifier!__. 

* Keep the score function at `f1_micro`, otherwise, we will be comparing 'æbler og pærer'. 

* And, you may also want to scale your input data for some models to perform better.

* __REMEMBER__, DO NOT USE any Neural Network models. This also means not to use any `Keras` or `Tensorflow` models...since they outperform most other models, and there are also too many examples on the internet to cut-and-paste from!

Check your result by printing the first _return_ value from `FullReport()` 
```python 
b1, m1 = FullReport(random_tuned , X_test, y_test, time_randomsearch)
print(b1)
```
that will display a result like
```
best: dat=mnist, score=0.90780, model=SGDClassifier(alpha=1.0,eta0=0.0001,learning_rate='invscaling')
```
and paste your currently best model into the message box, for ITMAL group 09 like
```
Grp09: best: dat=mnist, score=0.90780, model=SGDClassifier(alpha=1.0,eta0=0.0001,learning_rate='invscaling')

Grp09: CTOR for best model: SGDClassifier(alpha=1.0, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
              learning_rate='invscaling', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
```
              
on Brightspace: "L09: Regularisering, optimering og søgning" | "Qd MNIST Search Quest"

> https://itundervisning.ase.au.dk/itmal_quest/index.php

and, check if your score (for MNIST) is better than the currently best score. Republish if you get a better score than your own previously best. Deadline for submission of scores is the same as the deadline for the O3 journal handin.

Remember to provide an ITMAL group name manually, so we can identify a winner: the 1. st price is  cake! 

For the journal hand-in, report your progress in scoring choosing different models, hyperparameters to search and how you might need to preprocess your data...and note, that the journal will not be accepted unless it contains information about Your results published on the Brightspace 'Search Quest II' page!

In [ ]:
# TODO:(in code and text..)
assert False, "participate in the Search Quest---remember to publish your result(s) on Brightspace."